In [ ]:
from google.colab import files
files.upload()

!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tanlikesmath/diabetic-retinopathy-resized

100% 7.24G/7.25G [02:46<00:00, 36.1MB/s]
100% 7.25G/7.25G [02:46<00:00, 46.8MB/s]


In [ ]:
!unzip -q '/content/diabetic-retinopathy-resized.zip' -d data

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

In [ ]:
# The 'resized_train_cropped' dir contains black border cropped images.
# This script will first resize every (.jpg) image under that dir and then apply CLAHE and save it in-place.

import cv2
import os
os.chdir(r'/content/data/resized_train_cropped/resized_train_cropped')
i = os.walk(os.getcwd())
d = (256,256)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

for a,b,c in i:
  for z in range(len(c)):
      path = os.path.join(a,c[z])
      #print(path)
      image = cv2.imread(path)
      image = cv2.resize(image, d, interpolation = cv2.INTER_AREA)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      image = clahe.apply(image)
      cv2.imwrite(path, image)

In [ ]:
traindf=pd.read_csv('/content/datasets_131128_418031_trainLabels_cropped.csv',dtype=str)

In [ ]:
traindf.head()

,Unnamed: 0,Unnamed: 0.1,image,level
0,0,0,10_left,0
1,1,1,10_right,0
2,2,2,13_left,0
3,3,3,13_right,0
4,4,4,15_left,1


In [ ]:
!nvidia-smi

In [ ]:
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPool2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Layer, Input, MaxPooling2D, ReLU, Add, GlobalAveragePooling2D, Dense

In [ ]:
inputs = Input(shape = (256, 256, 3))

x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)


output = Dense(units=5, activation='softmax')(x)

model = Model(inputs=inputs, outputs=output)

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 256, 256, 16)      2320      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
separable_conv2d_8 (Separabl (None, 128, 128, 32)      688       
_________________________________________________________________
separable_conv2d_9 (Separabl (None, 128, 128, 32)      1344      
_________________________________________________________________
batch_normalization_4 (Batch (None, 128, 128, 32)      128 

In [ ]:
# Appending (.jpeg) ext. to every image name in the dataframe
def append_ext(fn):
    return fn+".jpeg"

traindf["image"] = traindf["image"].apply(append_ext)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)

# Flow from dataframe

train_generator = datagen.flow_from_dataframe(
dataframe = traindf,
directory = "/content/data/resized_train_cropped/resized_train_cropped",
x_col = "image",
y_col = "level",
subset = "training",
batch_size = 32,
seed = 42,
shuffle = True,
class_mode = "categorical",
target_size = (256,256))

valid_generator = datagen.flow_from_dataframe(
dataframe = traindf,
directory = "/content/data/resized_train_cropped/resized_train_cropped",
x_col = "image",
y_col = "level",
subset = "validation",
batch_size = 32,
seed = 42,
shuffle = True,
class_mode = "categorical",
target_size = (256,256))

Found 26331 validated image filenames belonging to 5 classes.
Found 8777 validated image filenames belonging to 5 classes.


In [ ]:
model.compile(optimizer = 'adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
save_weights_path = '/content/weights'
if not os.path.exists(save_weights_path):
  os.mkdir(save_weights_path)

cpk_path = save_weights_path+'/weights_{epoch:03d}-{val_accuracy:.4f}.h5'

In [ ]:
 checkpoint = ModelCheckpoint(
  filepath=cpk_path,
  monitor='val_accuracy',
  mode='max',
  verbose=1,
  save_best_only=True,
  save_weights_only=True,
  save_freq='epoch'
  )

reducelr = ReduceLROnPlateau(
  monitor='val_loss',
  factor=0.2,
  patience=5,
  verbose=1
  )

callbacks = [checkpoint, reducelr]

In [ ]:
batch_size = 32
history = model.fit_generator(train_generator, steps_per_epoch = train_generator.samples // batch_size,
                              epochs = 10,
                              validation_data = valid_generator,
                              validation_steps = valid_generator.samples // batch_size,
                              callbacks=[checkpoint, reducelr]
                              )

Epoch 1/10
822/822 [==============================] - ETA: 0s - loss: 0.9106 - accuracy: 0.7361
Epoch 00001: val_accuracy improved from -inf to 0.72776, saving model to /content/weights/weights_001-0.7278.h5
822/822 [==============================] - 267s 325ms/step - loss: 0.9106 - accuracy: 0.7361 - val_loss: 0.8898 - val_accuracy: 0.7278 - lr: 0.0010
Epoch 2/10
822/822 [==============================] - ETA: 0s - loss: 0.8860 - accuracy: 0.7371
Epoch 00002: val_accuracy improved from 0.72776 to 0.72787, saving model to /content/weights/weights_002-0.7279.h5
822/822 [==============================] - 267s 325ms/step - loss: 0.8860 - accuracy: 0.7371 - val_loss: 0.8841 - val_accuracy: 0.7279 - lr: 0.0010
Epoch 3/10
822/822 [==============================] - ETA: 0s - loss: 0.8741 - accuracy: 0.7374
Epoch 00003: val_accuracy did not improve from 0.72787
822/822 [==============================] - 267s 325ms/step - loss: 0.8741 - accuracy: 0.7374 - val_loss: 0.8763 - val_accuracy: 0.7279